In [10]:
import pandas as pd
from datetime import  date, timedelta
import time
import unicodedata
import string
import requests
import mysql.connector
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

from sshtunnel import SSHTunnelForwarder
import pymysql as db
import pandas as pd
import datetime
import sshtunnel
import os
# .env ファイルをロードして環境変数へ反映
from dotenv import load_dotenv
load_dotenv('.env')
# 環境変数を参照

def removal_text(text:str) -> str:
    text = unicodedata.normalize("NFKC", text)
    text = text.translate(str.maketrans( '', '',string.punctuation  + '！'+ '　'+ ' '+'・'+'～' + '‐'))
    return text

def post_line_text(message:str,token:str) -> None:
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload) 

def post_line_text_and_image(message:str,image_path:str,token:str) -> None:
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    print('image_path',image_path)
    files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload,files=files) 

def insert_data_bulk(df:pd.DataFrame,cnx) -> None:
    insert_sql = f"""INSERT INTO {os.getenv('WORDPRESS_DB_TABLE')}(店舗名, 日付, Nのつく日, 都道府県, 機種名, 台番号, G数, 差枚, BB, RB,ART, BB確率, RB確率, ART確率, 合成確率) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    cur = cnx.cursor()
    cur.executemany(insert_sql, df.values.tolist())
    cnx.commit()
    print("Insert bulk data")

def delete_data(cnx,day:int) -> None:
    cursor = cnx.cursor()
    target_days_ago = datetime.date.today() - datetime.timedelta(days=day)
    target_days_ago_str = target_days_ago.strftime('%Y-%m-%d')
    target_days_ago_str
    sql = f"DELETE FROM {os.getenv('WORDPRESS_DB_TABLE')} WHERE 日付 < '{target_days_ago_str} 00:00:00';"
    cursor.execute(sql)
    cnx.commit()
    
def query_prefecture_parlar_data(cnx,prefecture_name:str , past_target_day_number:int) -> pd.DataFrame :
    past_target_day:date = datetime.date.today() - timedelta(days=past_target_day_number)
    str_past_target_day:str = past_target_day.strftime('%Y-%m-%d')
    # SQLを実行する
    #SELECT 引っ張ってきたい列名　FROM　テーブル名 WHERE 条件列 = 'ジャグ（文字列の完全一致）' " 
    column_name = [ '店舗名','日付','Nのつく日', '都道府県', '機種名', '台番号', 'G数', '差枚', \
                    'BB', 'RB', 'ART', 'BB確率','RB確率', 'ART確率', '合成確率', 'id']
    print(column_name)
    with cnx.cursor() as cursor:
        sql = f"SELECT * FROM {os.getenv('WORDPRESS_DB_TABLE')} WHERE 都道府県 = '{prefecture_name}' AND 日付 = '{str_past_target_day} 00:00:00' ;" #AND 機種名 = '主役は銭形3'
        cursor.execute(sql)
        # # Select結果を取り出す
        #results = cursor.fetchall()
        print(sql)
        past_target_date_prefecture_parlar_data_df = pd.DataFrame(data=cursor.fetchall(), index = None, columns = column_name)
    return past_target_date_prefecture_parlar_data_df

In [13]:
with sshtunnel.SSHTunnelForwarder(
    (os.getenv('SSH_USERNAME'), 10022), 
    ssh_username="pachislot777", 
    ssh_private_key_password=os.getenv('SSH_PRIVATE_KEY_PASSWORD'), 
    ssh_pkey=r"sercret\akasaka.key", 
    remote_bind_address=("mysql8055.xserver.jp", 3306 )
    ) as server:

    # SSH接続確認
    print(f"local bind port: {server.local_bind_port}")
    # データベース接続
    cnx = mysql.connector.connect(
        host="localhost", 
        port=server.local_bind_port, 
        user=os.getenv('WORDPRESS_DB_ID'), 
        password=os.getenv('DB_PASSWORD'), 
        database=os.getenv('WORDPRESS_DB_NAME'), 
        charset="utf8",
        use_pure=True
        )

    # 接続確認
    print(f"sql connection status: {cnx.is_connected()}")
    cursor = cnx.cursor()
    past_target_date_prefecture_parlar_data_df = query_prefecture_parlar_data(cnx,'愛知県', 2)
    # 終了
    display(past_target_date_prefecture_parlar_data_df)
    cnx.close()

local bind port: 61556
sql connection status: True
['店舗名', '日付', 'Nのつく日', '都道府県', '機種名', '台番号', 'G数', '差枚', 'BB', 'RB', 'ART', 'BB確率', 'RB確率', 'ART確率', '合成確率', 'id']
SELECT * FROM parlar_data WHERE 都道府県 = '愛知県' AND 日付 = '2023-04-05 00:00:00' ;


,店舗名,日付,Nのつく日,都道府県,機種名,台番号,G数,差枚,BB,RB,ART,BB確率,RB確率,ART確率,合成確率,id
0,100℃大曽根店,2023-04-05,5,愛知県,アイムジャグラーEX,367,45,-74,0,0,0,1/0.0,1/0.0,,1/0.0,95865
1,100℃大曽根店,2023-04-05,5,愛知県,アイムジャグラーEX,368,34,-61,0,0,0,1/0.0,1/0.0,,1/0.0,95866
2,100℃大曽根店,2023-04-05,5,愛知県,アイムジャグラーEX,369,0,-1,0,0,0,1/0.0,1/0.0,,1/0.0,95867
3,100℃大曽根店,2023-04-05,5,愛知県,アイムジャグラーEX,392,130,-127,0,0,0,1/0.0,1/0.0,,1/0.0,95868
4,100℃大曽根店,2023-04-05,5,愛知県,アイムジャグラーEX,393,152,-200,0,0,0,1/0.0,1/0.0,,1/0.0,95869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38223,東宝プラザ野並店,2023-04-05,5,愛知県,OVERSLOT「AINZOOALGOWN絶対支配者光臨」,760,0,0,0,0,0,1/0.0,1/0.0,,1/0.0,134088
38224,東宝プラザ野並店,2023-04-05,5,愛知県,TIGERBUNNY,761,0,0,0,0,0,1/0.0,1/0.0,,1/0.0,134089
38225,東宝プラザ野並店,2023-04-05,5,愛知県,とある科学の超電磁砲,762,1987,-1208,0,7,0,1/0.0,1/283.9,,1/283.9,134090
38226,東宝プラザ野並店,2023-04-05,5,愛知県,かまいたちの夜,763,0,0,0,0,0,1/0.0,1/0.0,,1/0.0,134091


In [16]:
def generate_processed_kisyubetu_df(_df):
    kisyubetu_master_df = _df.groupby('機種名').sum()
    kisyubetu_master_df['総台数'] = _df.groupby('機種名').size()
    kisyubetu_master_df = kisyubetu_master_df.reset_index(drop=False).reset_index().rename(columns={'index': '機種順位','ゲーム数': 'G数'})
    kisyubetu_master_df['機種順位'] = kisyubetu_master_df['機種順位'] + 1
    kisyubetu_master_df[['機種順位','機種名','総台数','G数','差枚']]
    kisyubetu_win_daissuu_list = []
    kisyubetu_master_df_list = []
    for kisyu_name in kisyubetu_master_df['機種名']:
        kisyu_df = _df.query('機種名 == @kisyu_name')
        kisyubetu_master_df_list.append(kisyu_df)
        kisyu_win_daisuu = len(kisyu_df[kisyu_df['差枚'] > 0])
        kisyubetu_win_daissuu_list.append(kisyu_win_daisuu)
    kisyubetu_master_df['勝率'] = kisyubetu_win_daissuu_list
    kisyubetu_master_df['勝率'] = kisyubetu_master_df['勝率'].astype(str)
    kisyubetu_master_df['総台数'] = kisyubetu_master_df['総台数'].astype(int)
    kisyubetu_master_df['平均G数'] = kisyubetu_master_df['G数'] / kisyubetu_master_df['総台数']
    kisyubetu_master_df['平均G数'] = kisyubetu_master_df['平均G数'].astype(int)
    kisyubetu_master_df = kisyubetu_master_df[kisyubetu_master_df['総台数'] > 2 ]
    kisyubetu_master_df['差枚'] = kisyubetu_master_df['差枚'].astype(int)
    kisyubetu_master_df['平均差枚'] = kisyubetu_master_df['差枚'] / kisyubetu_master_df['総台数']
    kisyubetu_master_df['平均差枚'] = kisyubetu_master_df['平均差枚'].astype(int)
    kisyubetu_master_df['総台数'] = kisyubetu_master_df['総台数'].astype(str)
    kisyubetu_master_df['勝率'] = kisyubetu_master_df['勝率'] + '/' + kisyubetu_master_df['総台数']
    kisyubetu_master_df['勝率'] = kisyubetu_master_df['勝率'].map(lambda x : '(' + x + '台) ' + str(round(int(x.split('/')[0])/int(x.split('/')[1])*100,1))  + '%')
    kisyubetu_master_df = kisyubetu_master_df[['機種順位','機種名','勝率','総台数','G数','平均G数','差枚','平均差枚']]
    kisyubetu_master_df = kisyubetu_master_df.sort_values('平均差枚',ascending=False)
    kisyubetu_master_df['機種順位'] = list(range(1,len(kisyubetu_master_df)+1))
    return kisyubetu_master_df

In [19]:
twitter = UtilsTwitterClass()

In [74]:
prefecture ='愛知県'
all_text =''
extract_prefecture_tenpo_data_df = past_target_date_prefecture_parlar_data_df
tenpobetsu_all_tenpo_df =  extract_prefecture_tenpo_data_df.groupby('店舗名').sum()
tenpobetsu_all_tenpo_df['総台数'] =  extract_prefecture_tenpo_data_df.groupby('店舗名').size()
tenpobetsu_all_tenpo_df['平均G数'] = tenpobetsu_all_tenpo_df['G数'] / tenpobetsu_all_tenpo_df['総台数']
tenpobetsu_all_tenpo_df['平均G数'] = tenpobetsu_all_tenpo_df['平均G数'].astype(int)
tenpobetsu_all_tenpo_df['平均差枚'] = tenpobetsu_all_tenpo_df['差枚'] / tenpobetsu_all_tenpo_df['総台数']
tenpobetsu_all_tenpo_df['平均差枚'] = tenpobetsu_all_tenpo_df['平均差枚'].astype(int)
tenpobetsu_all_tenpo_df = tenpobetsu_all_tenpo_df.sort_values('平均差枚',ascending=False)
tenpobetsu_all_tenpo_df = tenpobetsu_all_tenpo_df.reset_index()
tenpobetsu_all_tenpo_df['店舗出率'] =(((tenpobetsu_all_tenpo_df['G数'] * 3) + tenpobetsu_all_tenpo_df['差枚']) / (tenpobetsu_all_tenpo_df['G数'] * 3) )*100
tenpobetsu_all_tenpo_df['店舗出率'] = tenpobetsu_all_tenpo_df['店舗出率'].map(lambda x : round(x,1))
for i,(index , record) in enumerate(tenpobetsu_all_tenpo_df.iterrows()):
    #print(record['店舗名'])
    tenpo_zendai_df = past_target_date_prefecture_parlar_data_df[past_target_date_prefecture_parlar_data_df['店舗名'] == record['店舗名']]
    kisyubetsu_df = generate_processed_kisyubetu_df(tenpo_zendai_df)
    if int(record['平均差枚']) > 0:
        heikin_samai = '+' + str(record['平均差枚'])
    else:
        heikin_samai =  str(record['平均差枚'])#[su_spoiler title="4/6(木)愛知県 1位/159店舗　<br>◆プレイランドキャッスル大曽根 <br>5331G +63枚" style="fancy" icon="chevron-circle" anchor="Hello"]
    tweet_text =f'''[su_spoiler title="{twitter.yesterday_date_string}{prefecture} {i+1}位/{len(tenpobetsu_all_tenpo_df.index.unique())}店舗
{record['店舗名'].replace('店','')} 
平均G数 {record['平均G数']}G　平均差枚 {heikin_samai}枚
出率 {record['店舗出率']}%" style="fancy" icon="chevron-circle" anchor="Hello"]\n'''
    kisyu_count = 0
    gaiyou_df = pd.DataFrame({
            '平均G数': f'{record["平均G数"]}G',
            '店舗平均差枚': f'{heikin_samai}枚',
            '店舗出立': f'{record["店舗出率"]}%',
            '全体勝率': f'{len(tenpo_zendai_df[tenpo_zendai_df["差枚"] > 0])}/{str(record["総台数"])}',
            '1000枚↑': f'{str(len(tenpo_zendai_df[tenpo_zendai_df.差枚 > 1000]))}/{record["総台数"]}({round(len(tenpo_zendai_df[tenpo_zendai_df.差枚 > 1000])/record["総台数"]*100,1)}%)',
            '3000枚↑': f'{str(len(tenpo_zendai_df[tenpo_zendai_df.差枚 > 3000]))}/{record["総台数"]}({round(len(tenpo_zendai_df[tenpo_zendai_df.差枚 > 3000])/record["総台数"]*100,1)}%)'},index=[0]).T
    gaiyou_df.columns = [f'概要']
    display(gaiyou_df)
    #print(gaiyou_df.to_html(header=None))
    tweet_text += gaiyou_df.to_html(header=None) + '\n'
    # for _,kisyu_betsu_record in kisyubetsu_df.iterrows():
    #     tweet_text += f"\n {kisyu_count+1}位 ▼{kisyu_betsu_record['機種名']}{kisyu_betsu_record['勝率']}\n    ・平均差枚 +{kisyu_betsu_record['平均差枚']}枚 平均G数 {kisyu_betsu_record['平均G数']}G"
    #     kisyu_count += 1
    #     if kisyu_count >= 5:
    #         break
    tweet_text += kisyubetsu_df[['機種名','平均G数','平均差枚','勝率']][:10].to_html(index=False)
    tweet_text += '\n[/su_spoiler]'
    #post_line_text(tweet_text,line_area_token[prefecture])
    if i >= 5:
        break
    
    all_text += tweet_text
print(all_text)
#break

,概要
平均G数,2855G
店舗平均差枚,+213枚
店舗出立,102.5%
全体勝率,192/392
1000枚↑,78/392(19.9%)
3000枚↑,22/392(5.6%)


,概要
平均G数,5531G
店舗平均差枚,+63枚
店舗出立,100.4%
全体勝率,168/359
1000枚↑,95/359(26.5%)
3000枚↑,33/359(9.2%)


,概要
平均G数,843G
店舗平均差枚,0枚
店舗出立,100.0%
全体勝率,0/224
1000枚↑,0/224(0.0%)
3000枚↑,0/224(0.0%)


,概要
平均G数,1351G
店舗平均差枚,0枚
店舗出立,100.0%
全体勝率,0/214
1000枚↑,0/214(0.0%)
3000枚↑,0/214(0.0%)


,概要
平均G数,1140G
店舗平均差枚,0枚
店舗出立,100.0%
全体勝率,0/280
1000枚↑,0/280(0.0%)
3000枚↑,0/280(0.0%)


,概要
平均G数,2629G
店舗平均差枚,0枚
店舗出立,100.0%
全体勝率,176/480
1000枚↑,82/480(17.1%)
3000枚↑,15/480(3.1%)


[su_spoiler title="4/6(木)愛知県 1位/159店舗
プレイランドキャッスル大曽根 
平均G数 2855G　平均差枚 +213枚
出率 102.5%" style="fancy" icon="chevron-circle" anchor="Hello"]
<table border="1" class="dataframe">
  <tbody>
    <tr>
      <th>平均G数</th>
      <td>2855G</td>
    </tr>
    <tr>
      <th>店舗平均差枚</th>
      <td>+213枚</td>
    </tr>
    <tr>
      <th>店舗出立</th>
      <td>102.5%</td>
    </tr>
    <tr>
      <th>全体勝率</th>
      <td>192/392</td>
    </tr>
    <tr>
      <th>1000枚↑</th>
      <td>78/392(19.9%)</td>
    </tr>
    <tr>
      <th>3000枚↑</th>
      <td>22/392(5.6%)</td>
    </tr>
  </tbody>
</table>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>機種名</th>
      <th>平均G数</th>
      <th>平均差枚</th>
      <th>勝率</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>スマスロ北斗の拳</td>
      <td>6888</td>
      <td>1690</td>
      <td>(9/12台) 75.0%</td>
    </tr>
    <tr>
      <td>革命機ヴァルヴレイヴ</td>
      <td>1983</td>
      <td>989</td>
      <td>(6/15台) 40.0%</td>
    

In [32]:
kisyubetsu_df[['機種名','平均G数','平均差枚','勝率']][:10]

,機種名,平均G数,平均差枚,勝率
26,犬夜叉,3493,1404,(2/3台) 66.7%
24,機動戦士ガンダムユニコーン,4402,1271,(4/6台) 66.7%
9,ハナハナホウオウ天翔30,6047,704,(56/87台) 64.4%
5,スマスロ北斗の拳,6954,554,(11/24台) 45.8%
13,ファンキージャグラー2,6137,359,(11/16台) 68.8%


In [33]:
print(kisyubetsu_df[['機種名','平均G数','平均差枚','勝率']][:10].to_html(index=False))

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>機種名</th>
      <th>平均G数</th>
      <th>平均差枚</th>
      <th>勝率</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>犬夜叉</td>
      <td>3493</td>
      <td>1404</td>
      <td>(2/3台) 66.7%</td>
    </tr>
    <tr>
      <td>機動戦士ガンダムユニコーン</td>
      <td>4402</td>
      <td>1271</td>
      <td>(4/6台) 66.7%</td>
    </tr>
    <tr>
      <td>ハナハナホウオウ天翔30</td>
      <td>6047</td>
      <td>704</td>
      <td>(56/87台) 64.4%</td>
    </tr>
    <tr>
      <td>スマスロ北斗の拳</td>
      <td>6954</td>
      <td>554</td>
      <td>(11/24台) 45.8%</td>
    </tr>
    <tr>
      <td>ファンキージャグラー2</td>
      <td>6137</td>
      <td>359</td>
      <td>(11/16台) 68.8%</td>
    </tr>
    <tr>
      <td>沖ドキGOLD30</td>
      <td>4863</td>
      <td>324</td>
      <td>(17/36台) 47.2%</td>
    </tr>
    <tr>
      <td>この素晴らしい世界に祝福を</td>
      <td>4519</td>
      <td>170</td>
      <td>(1/3台) 33.3%</td>
    </tr>
    <tr>
      <

In [ ]:
平均G数 {record['平均G数']}G
店舗平均差枚 {heikin_samai}枚
店舗出率 {record['店舗出率']}%
全体勝率 {len(tenpo_zendai_df[tenpo_zendai_df['差枚'] > 0])}/{str(record['総台数'])}
1000枚↑ {str(len(tenpo_zendai_df[tenpo_zendai_df.差枚 > 1000]))}/{record['総台数']}({round(len(tenpo_zendai_df[tenpo_zendai_df.差枚 > 1000])/record['総台数']*100,1)}%)
3000枚↑ {str(len(tenpo_zendai_df[tenpo_zendai_df.差枚 > 3000]))}/{record['総台数']}({round(len(tenpo_zendai_df[tenpo_zendai_df.差枚 > 3000])/record['総台数']*100,1)}%)

In [61]:
gaiyou_df.index.set_names('概要')
gaiyou_df

Index(['平均G数', '店舗平均差枚', '店舗出立', '全体勝率', '1000枚↑', '3000枚↑'], dtype='object', name='概要')

In [71]:
gaiyou_df = pd.DataFrame({
            '平均G数': f'{record["平均G数"]}G',
            '店舗平均差枚': f'{heikin_samai}枚',
            '店舗出立': f'{record["店舗出率"]}%',
            '全体勝率': f'{len(tenpo_zendai_df[tenpo_zendai_df["差枚"] > 0])}/{str(record["総台数"])}',
            '1000枚↑': f'{str(len(tenpo_zendai_df[tenpo_zendai_df.差枚 > 1000]))}/{record["総台数"]}({round(len(tenpo_zendai_df[tenpo_zendai_df.差枚 > 1000])/record["総台数"]*100,1)}%)',
            '3000枚↑': f'{str(len(tenpo_zendai_df[tenpo_zendai_df.差枚 > 3000]))}/{record["総台数"]}({round(len(tenpo_zendai_df[tenpo_zendai_df.差枚 > 3000])/record["総台数"]*100,1)}%)'},index=[0]).T
gaiyou_df.columns = [f'概要']
display(gaiyou_df)
print(gaiyou_df.to_html(header=None))

,概要
平均G数,5531G
店舗平均差枚,+63枚
店舗出立,100.4%
全体勝率,168/359
1000枚↑,95/359(26.5%)
3000枚↑,33/359(9.2%)


<table border="1" class="dataframe">
  <tbody>
    <tr>
      <th>平均G数</th>
      <td>5531G</td>
    </tr>
    <tr>
      <th>店舗平均差枚</th>
      <td>+63枚</td>
    </tr>
    <tr>
      <th>店舗出立</th>
      <td>100.4%</td>
    </tr>
    <tr>
      <th>全体勝率</th>
      <td>168/359</td>
    </tr>
    <tr>
      <th>1000枚↑</th>
      <td>95/359(26.5%)</td>
    </tr>
    <tr>
      <th>3000枚↑</th>
      <td>33/359(9.2%)</td>
    </tr>
  </tbody>
</table>


In [1]:
print('ライブラリの読み込み完了')
try:
    from src.utils import *
except:
    from utils import *

ライブラリの読み込み完了
読み込み完了


In [2]:
class UtilsDataClass(Blog):
    def __init__(self):
        super().__init__()
    def another_method(self):
        print(f'b')

In [3]:
utilsdata = UtilsDataClass()

In [4]:
utilsdata.add_target_date(-2)

インスタンに日付:4月4日(火)など三つの変数が追加されました


In [5]:
dir(utilsdata)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_target_date',
 'another_method',
 'check_url',
 'create_main_text',
 'error_pledge_name_list',
 'generate_by_pledge_text',
 'generate_thumbnail',
 'get_post_list',
 'post_blog',
 'post_line',
 'target_date',
 'target_date_string_jp',
 'target_date_string_sql',
 'upload_image',
 'wp',
 'wp_tag_add',
 'wp_update_post']

In [6]:
vars(utilsdata)

{'error_pledge_name_list': [],
 'wp': <wordpress_xmlrpc.base.Client at 0x209d5829ac0>,
 'target_date': datetime.date(2023, 4, 4),
 'target_date_string_jp': '4月4日(火)',
 'target_date_string_sql': '2023-04-04'}

In [9]:
post_list : list[WordPressPost] = utilsdata.get_post_list()
for post in post_list:
    print(post)
    #break

【4/11(火)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
【三重県】4月10日(月) パチンコスロットイベント取材まとめ
【福井県】4月10日(月) パチンコスロットイベント取材まとめ
【石川県】4月10日(月) パチンコスロットイベント取材まとめ
【富山県】4月10日(月) パチンコスロットイベント取材まとめ
【新潟県】4月10日(月) パチンコスロットイベント取材まとめ
【岐阜県】4月10日(月) パチンコスロットイベント取材まとめ
【長野県】4月10日(月) パチンコスロットイベント取材まとめ
【山梨県】4月10日(月) パチンコスロットイベント取材まとめ
【静岡県】4月10日(月) パチンコスロットイベント取材まとめ
【愛知県】4月10日(月) パチンコスロットイベント取材まとめ
【石川県】4月5日(水) パチンコスロットイベント取材まとめ
【富山県】4月5日(水) パチンコスロットイベント取材まとめ
【新潟県】4月5日(水) パチンコスロットイベント取材まとめ
【岐阜県】4月5日(水) パチンコスロットイベント取材まとめ
【長野県】4月5日(水) パチンコスロットイベント取材まとめ
【山梨県】4月5日(水) パチンコスロットイベント取材まとめ
【静岡県】4月5日(水) パチンコスロットイベント取材まとめ
【愛知県】4月5日(水) パチンコスロットイベント取材まとめ
【4/5(水)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
【4/10(月)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
【三重県】4月9日(日) パチンコスロットイベント取材まとめ
【福井県】4月9日(日) パチンコスロットイベント取材まとめ
【石川県】4月9日(日) パチンコスロットイベント取材まとめ
【富山県】4月9日(日) パチンコスロットイベント取材まとめ
【新潟県】4月9日(日) パチンコスロットイベント取材まとめ
【岐阜県】4月9日(日) パチンコスロットイベント取材まとめ
【長野県】4月9日(日) パチンコスロットイベント取材まとめ
【山梨県】4月9日(日) パチンコスロットイベント取材まとめ
【静岡県】4月9日(日) パチンコスロットイベント取材まとめ
【三重県】4月8日(土) パチンコスロットイベント取材まとめ


In [ ]:
for target_day_number in range(0,6):
    scraping.add_target_date(target_day_number)
    browser = scraping.login_scraping_site('chubu')
    prefecture_name_and_number_dict = scraping.get_prefecture_name_and_number_dict()
    for prefecture_name in prefecture_name_and_number_dict:
        blog = Blog()
        print(prefecture_name,target_day_number)
        blog.add_target_date (target_day_number)
        blog.prefecture_name = prefecture_name
        print(prefecture_name)
        scraping_target_date_pledge_df = scraping.read_target_date_pledge_df(prefecture_name)
        read_convert_parlar_name_df = scraping.read_convert_parlar_name_df()
        merged_syuzai_pledge_df = scraping.generate_merged_syuzai_pledge_df()
        save_main_image_path_list = scraping.create_pledge_main_images()
        title = f"【{blog.prefecture_name}】{blog.target_date_string_jp } パチンコスロットイベント取材まとめ"
        if title in post_title_contentid_dict:
            update_content_id:int = int(post_title_contentid_dict[title])
            print('既存の記事を更新します',update_content_id)
            files:list[Client] = blog.wp.call(methods.media.GetMediaLibrary({"parent_id": update_content_id}))
            for file in files:
                print('画像削除',file.id, file.title)
                ret = blog.wp.call(methods.posts.DeletePost(file.id))
                print(ret)
                #break